# Lab 2 trie oraz drzewo sufiksów - Jakub Janicki

In [1]:
import tabulate
import time

txt1 = "bbbd$"
txt2 = "aabbabd$"
txt3 = "ababcd$"
txt4 = "abcbccd$"
f = open("assets/1997_714_head.txt", encoding='utf8')
txt5 = f.read()

### Trie

In [2]:
class Leaf:
    def __init__(self, char):
        self.char = char
        self.kids = dict()

    def add_kid(self, char):
        kid = Leaf(char)
        self.kids[char] = kid
        return kid

    def have_kid(self, char):
        return char in self.kids

    def find_kid(self, char):
        return self.kids[char]

    def print(self):
        for char in self.kids.keys():
            print(char)
            self.kids[char].print()


class Trie:
    def __init__(self, pattern):
        self.root = Leaf("root")
        self.__add__(pattern)

    def print(self):
        self.root.print()

    def __add__(self, pattern):
        for i in range(len(pattern)):
            suffix = pattern[i:]
            leaf = self.root
            for char in suffix:
                if not leaf.have_kid(char):
                    leaf = leaf.add_kid(char)
                else:
                    leaf = leaf.kids.get(char)

    def __contains__(self, pattern):
        leaf = self.root
        for char in pattern:
            if char in leaf.kids:
                leaf = leaf.kids.get(char)
            else:
                return False
        return True

### Drzewo sufixów

In [3]:
class LeafSuffixTree:
    def __init__(self, string_start, string_end, end_idx=0):
        self.string_start = string_start
        self.string_end = string_end
        self.children = dict()
        self.end_idx = end_idx

    def addChildren(self, string_start, string_end, end_idx=0):
        self.children[(string_start, string_end)] = LeafSuffixTree(string_start, string_end, end_idx)
        return self.children[string_start, string_end]

    def __len__(self):
        return self.string_end - self.string_start + 1


class SuffixTree:
    def __init__(self, pattern):
        self.root = LeafSuffixTree(-1, -1)
        self.pattern = pattern
        self.len = len(pattern)

        for i in range(len(pattern)):
            self.__add__(i)

    def __add__(self, suffix_start):
        suffix_idx = suffix_start
        leaf = self.root
        while suffix_idx < self.len:
            children = None
            if not len(leaf.children):
                leaf.addChildren(suffix_idx, self.len - 1)
                return

            for child_idx, child in enumerate(list(leaf.children.values())):
                if self.pattern[child.string_start] == self.pattern[suffix_idx]:
                    children = child
                    break
                if child_idx == len(leaf.children) - 1:
                    leaf.addChildren(suffix_idx, self.len - 1)
                    return

            i = 0
            between = children
            while i < len(children):

                if self.pattern[children.string_start + i] != self.pattern[suffix_idx + i]:
                    leaf.children.pop((children.string_start, children.string_end))
                    between = leaf.addChildren(children.string_start, children.string_start + i - 1)
                    children.string_start = children.string_start + i
                    between.children[(children.string_start, children.string_end)] = children
                    break
                i += 1
            suffix_idx += i
            leaf = between

    def __contains__(self, pattern):
        leaf = self.root
        i = 0
        flag = True
        while i < len(pattern) and flag:
            flag = False
            for child in leaf.children.values():
                if self.pattern[child.string_start] == pattern[i]:
                    flag = True
                    if self.pattern[child.string_start:child.string_end+1] == pattern[i:i + len(child)]:
                        i += len(child)
                        leaf = child
                        break
                    else:
                        return False
        return flag

    def print(self, leaf):
        if leaf.end_idx:
            return
        for l in leaf.children.values():
            print(self.pattern[l.string_start: l.string_end + 1])
            self.print(l)
        print("-")

In [4]:
s=SuffixTree("aabbabd")
s.print(s.root)

a
abbabd
-
b
babd
-
d
-
-
-
b
babd
-
abd
-
d
-
-
d
-
-


In [5]:
def is_correct(structure, txt):
    for i in range(len(txt)):
        if not txt[i:] in structure:
            return False
    return True            

In [6]:
print("Test txt1, trie structure is correct: ", is_correct(Trie(txt1),txt1))
print("Test txt2, trie structure is correct: ", is_correct(Trie(txt2),txt2))
print("Test txt3, trie structure is correct: ", is_correct(Trie(txt3),txt3))
print("Test txt4, trie structure is correct: ", is_correct(Trie(txt4),txt4))
print("Test txt5, trie structure is correct: ", is_correct(Trie(txt5),txt5))

Test txt1, trie structure is correct:  True
Test txt2, trie structure is correct:  True
Test txt3, trie structure is correct:  True
Test txt4, trie structure is correct:  True
Test txt5, trie structure is correct:  True


In [7]:
print("Test txt1, sufix tree structure is correct: ", is_correct(SuffixTree(txt1),txt1))
print("Test txt2, sufix tree structure is correct: ", is_correct(SuffixTree(txt2),txt2))
print("Test txt3, sufix tree structure is correct: ", is_correct(SuffixTree(txt3),txt3))
print("Test txt4, sufix tree structure is correct: ", is_correct(SuffixTree(txt4),txt4))
print("Test txt5, sufix tree structure is correct: ", is_correct(SuffixTree(txt5),txt5))

Test txt1, sufix tree structure is correct:  True
Test txt2, sufix tree structure is correct:  True
Test txt3, sufix tree structure is correct:  True
Test txt4, sufix tree structure is correct:  True
Test txt5, sufix tree structure is correct:  True


In [8]:
def timer(func, text):
    start = time.time()
    structure = func(text)
    end = time.time()
    return int(round((end - start)*(10**6),0))


def time_comparator():
    table = [["trie",timer(Trie,txt1),timer(Trie,txt2),timer(Trie,txt3),timer(Trie,txt4),timer(Trie,txt5)]]
    table.append(["suffix tree", timer(SuffixTree,txt1),timer(SuffixTree,txt2),timer(SuffixTree,txt3),timer(SuffixTree,txt4),timer(SuffixTree,txt5)])
    print(tabulate.tabulate(table,headers=["structure", "txt1","txt2","txt3","txt4","txt5"], tablefmt="fancy_grid"))


time_comparator()

╒═════════════╤════════╤════════╤════════╤════════╤══════════╕
│ structure   │   txt1 │   txt2 │   txt3 │   txt4 │     txt5 │
╞═════════════╪════════╪════════╪════════╪════════╪══════════╡
│ trie        │    997 │      0 │      0 │      0 │ 10240187 │
├─────────────┼────────┼────────┼────────┼────────┼──────────┤
│ suffix tree │      0 │      0 │      0 │      0 │   126830 │
╘═════════════╧════════╧════════╧════════╧════════╧══════════╛


In [9]:
def timer2(func, text):
    structure = func(text)
    start = time.time()
    for i in range(len(text)):
        text[i:] in structure
    end = time.time()
    return int(round((end - start)*(10**6),0))


def time_comparator():
    table = [["trie",timer2(Trie,txt5)]]
    table.append(["suffix tree", timer2(SuffixTree,txt5)])
    print(tabulate.tabulate(table,headers=["structure","txt5"], tablefmt="fancy_grid"))


time_comparator()

╒═════════════╤═════════╕
│ structure   │    txt5 │
╞═════════════╪═════════╡
│ trie        │ 1344403 │
├─────────────┼─────────┤
│ suffix tree │  153589 │
╘═════════════╧═════════╛
